In [ ]:
from pyspark import SparkContext
import pyspark

conf = pyspark.SparkConf().set("spark.driver.host", "localhost")
sc = SparkContext.getOrCreate()
sc.setLogLevel('OFF')

In [ ]:
supervisor_sf_pair = sc.textFile("../data/SF_business/supervisor_sf.tsv")\
                       .map(lambda x : x.split("\t"))\
                       .map(lambda x : (x[0], x[1]))
zip_biz_pair = sc.textFile("../data/SF_business/filtered_registered_business_sf.tsv")\
                 .map(lambda x : x.split("\t"))\
                 .map(lambda x : (x[0], x[1:]))

## 1. Using “filtered_registered_business_sf.tsv” and “supervisor_sf.tsv”, list unique business names without a supervisor ordered by name.

In [ ]:
zip_biz_pair.

## 2. Create pairs of business name and supervisor id ordered by supervisor id (first) and business name (second).

In [ ]:
zip_biz_pair.

## 3. Create pairs of business and supervisor id for all the business ordered by supervisor id (first) and business name (second). (although it may not have a district supervisor).

In [ ]:
zip_biz_pair.

In [ ]:
sc.stop()